In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline
from numpy.random import choice
# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix
import math
# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from metrics import precision_at_k
from implicit.evaluation import  mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [6]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
result = pd.read_csv('predictions_basic.csv')
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[867846, 223642, 873201, 1054917, 13190151]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]"


In [9]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()
def weighted_random_recommendation(items, items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items)
    recs = random.choices(items, weights=items_weights, k=n)
    
    return recs

In [10]:
%%time
weights = data.groupby(['item_id']).agg(sum)['sales_value'].to_list()

items = data.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

result.head(2)

Wall time: 6.17 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1105730, 47963, 847071, 7166673, 2386561]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[912997, 855192, 6029743, 964616, 879270]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [29]:
result.to_csv('predictions_basic_1.csv', index=False)

In [30]:
result = pd.read_csv('predictions_basic_1.csv')
result.head(3)



,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,[ 821867 834484 856942 865456 889248 ...,"[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1105730, 47963, 847071, 7166673, 2386561]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[912997, 855192, 6029743, 964616, 879270]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[867846, 223642, 873201, 1054917, 13190151]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]","[3363966, 12183254, 7026550, 5552733, 6548709]"


In [33]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(np.array(row[name_col][1:-2].split()), np.array(row['actual'][1:-2].split())), axis=1).mean(), 4)}:{name_col}")

1.0:actual
0.0:random_recommendation
0.0014:popular_recommendation
0.0001:itemitem
0.0002:cosine
0.0:tfidf
0.0006:own_purchases
0.0:weighted_random_recommendation


## Не знаю почему как во время вебинара, предполагаю, что проблема в типах данных и чтении csv, можете подсказать в чем проблема?

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Много проблем с библиотекой implicit, разобраться пока не смог поэтому и моделей нет. 